In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from api_key import g_key


# Store Part I results into DataFrame

In [2]:
# Retreive citites data from WeatherPy
cities_data_path = ("../WeatherPy/full_cities_data_df.csv")

cities_data_df = pd.read_csv(cities_data_path)

cities_data_df = cities_data_df.drop(columns=['Unnamed: 0'])

cities_data_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,east london,-33.0153,27.9116,297.15,78,20,2.06,ZA,1616248640
1,hohhot,40.8106,111.6522,270.15,54,20,7.00,CN,1616248561
2,buin,-33.7333,-70.7500,293.71,71,0,1.61,CL,1616248859
3,atuona,-9.8000,-139.0333,299.65,75,58,4.87,PF,1616248582
4,grand gaube,-20.0064,57.6608,300.93,81,1,3.13,MU,1616248859


# Humidity Heatmap

In [10]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_data_df[['Lat', 'Lng']]

rating = cities_data_df['Humidity'].astype(float)

# Add Heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                dissipating=False, max_intensity=10,
                                point_radius=2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

In [4]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness

ideal_weather = cities_data_df.loc[(cities_data_df['Max Temp'] <= 80) & (cities_data_df['Max Temp'] >= 70)]
ideal_weather = cities_data_df.loc[(cities_data_df['Wind Speed'] <= 10)]
ideal_weather = cities_data_df.loc[(cities_data_df['Cloudiness'] == 0)]

ideal_weather = ideal_weather.dropna()

ideal_weather.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,buin,-33.7333,-70.7500,293.71,71,0,1.61,CL,1616248859
6,vagur,61.4732,-6.8119,279.78,81,0,10.38,FO,1616248860
7,albany,42.6001,-73.9662,277.04,31,0,3.34,US,1616248860
18,surt,31.2089,16.5887,296.11,44,0,3.91,LY,1616248863
30,burns lake,54.2331,-125.7533,270.15,86,0,3.09,CA,1616248870


# Hotel Map

In [5]:
# Store into variable named hotel_df
hotel_df = ideal_weather

# Add a "Hotel Name" column to the DataFrame
hotel_df['Hotel Name'] = ""


In [6]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
    
for index, row in hotel_df.iterrows():
    city = row['City']
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Retrieving results for {city}")
    response = requests.get(base_url, params=params)
    
    response = response.json()
#     pprint(response)
    
    try:
        print(f"The closest hotel from {city} is {response['results'][0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        
    except (KeyError, IndexError):
        print(f"No hotel in nearby area...Skipping")
        
    print("------------------------------------------------------")


Retrieving results for buin
The closest hotel from buin is Paihuen.
------------------------------------------------------
Retrieving results for vagur
The closest hotel from vagur is The Scouts House Camping.
------------------------------------------------------
Retrieving results for albany
No hotel in nearby area...Skipping
------------------------------------------------------
Retrieving results for surt
The closest hotel from surt is الهادي ابوشعاله سرت.
------------------------------------------------------
Retrieving results for burns lake
The closest hotel from burns lake is Burns Lake Bed and Breakfast.
------------------------------------------------------
Retrieving results for busselton
The closest hotel from busselton is Observatory Guest House.
------------------------------------------------------
Retrieving results for sur
The closest hotel from sur is Sur Plaza Hotel.
------------------------------------------------------
Retrieving results for carnarvon
The closest h

No hotel in nearby area...Skipping
------------------------------------------------------
Retrieving results for sonoita
The closest hotel from sonoita is Hotel Nora.
------------------------------------------------------
Retrieving results for bilma
The closest hotel from bilma is Expeditions Ténére voyages.
------------------------------------------------------
Retrieving results for coihueco
The closest hotel from coihueco is Frutillares Coihueco.
------------------------------------------------------
Retrieving results for coquimbo
The closest hotel from coquimbo is Punto de encuentro.
------------------------------------------------------
Retrieving results for general roca
The closest hotel from general roca is Hotel y Casino del Río General Roca.
------------------------------------------------------
Retrieving results for pisco
The closest hotel from pisco is Hotel San Isidro Oficial.
------------------------------------------------------
Retrieving results for champerico
The c

In [7]:
# Display hotel name
hotel_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,buin,-33.7333,-70.7500,293.71,71,0,1.61,CL,1616248859,Paihuen
6,vagur,61.4732,-6.8119,279.78,81,0,10.38,FO,1616248860,The Scouts House Camping
7,albany,42.6001,-73.9662,277.04,31,0,3.34,US,1616248860,
18,surt,31.2089,16.5887,296.11,44,0,3.91,LY,1616248863,الهادي ابوشعاله سرت
30,burns lake,54.2331,-125.7533,270.15,86,0,3.09,CA,1616248870,Burns Lake Bed and Breakfast


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [9]:
# Add marker layer ontop of heat map
# Display figure

fig = gmaps.figure()

markers = gmaps.marker_layer(locations,
                            info_box_content = hotel_info)

fig.add_layer(markers)

fig


Figure(layout=FigureLayout(height='420px'))